# Summary table for *COMPAS* data

Here we extract the summary measures for *Adult* data for four classifiers: (1) baseline, (2) project, (3) reduction, and (4) SenSR.
For each of the classifiers we perform gradient flow attack with step size $\frac{65}{1000\times i^{2/3}},$ where $i$ is the current step number. We perform 200 steps of attacks on each data points. 

First, we load the summary data for all experiments, and compile it in a panda dataframe. 

In [1]:
import re
import pandas as pd
with open('summary2.out', 'r') as f:
    data = f.read()
entries = re.split('\n', data)[:-1]
entries_dict = [eval(e) for e in entries]
data = pd.DataFrame(entries_dict)
data['reject'] = data['pval'] < 0.05

In [2]:
lr = 7e-2
data_lr = data.loc[data['lr'] == lr]
data_lr = data_lr.rename(columns= {'average_odds_difference_gen': 'AOD-gen', 'average_odds_difference_race': 'AOD-race', 'statistical_parity_difference_gen': 'SPD-gen', 'statistical_parity_difference_race': 'SPD-race', 'equal_opportunity_difference_race': 'EOD-race', 'equal_opportunity_difference_gen': 'EOD-gen'})

We only extract the following measures: balanced accuracy, average odds difference for gender and race and lower bound and proportion of times the test being rejected. 

In [3]:
measure = ['bal_acc', 'AOD-gen', 'AOD-race', 'lb', 'reject']
agg_dict = dict()
for key in measure:
    agg_dict[key] = ['mean', 'std']
result = data_lr.groupby(['algo'], as_index=False).agg(agg_dict)

In [8]:
rows = ['sensr', 'reduction', 'baseline', 'project']
row_names = ['Baseline', 'Project', 'Reduction','SenSR']
colnames=['Balanced Acc', '$\\text{AOD}_{\\text{gen}}$', '$\\text{AOD}_{\\text{race}}$', '$T_n$', 'Rejection Prop']


In [9]:
for i, c in enumerate(measure):
    if c == 'reject':
        idx_best = (1-result[c]['mean']).idxmin()
        result[colnames[i]] = (result[c]['mean']).apply("{:.1f}".format)
        #result.at[idx_best, colnames[i]] = '\\textbf{' + result[colnames[i]][idx_best] + '}'
        
    elif c == 'bal_acc':
        idx_best = (result[c]['mean']).idxmax()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
    elif c == 'lb':
        idx_best = (result[c]['mean']).idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val
    else:
        idx_best = (result[c]['mean']).abs().idxmin()
        result[colnames[i]] = result[c]['mean'].apply("{:.3f}".format)+ '$\pm$' + result[c]['std'].apply("{:.3f}".format)
        best_val = result[colnames[i]][idx_best].split('$\pm$')
        best_val = '$\pm$'.join(['\\textbf{' + best_val[0] + '}', best_val[1]])
        #result.at[idx_best, colnames[i]] = best_val


In [10]:
ind = dict()
for i, expt in enumerate(row_names):
    ind[i] = expt
    res = result[colnames].rename(index=ind)
res

,Balanced Acc,$\text{AOD}_{\text{gen}}$,$\text{AOD}_{\text{race}}$,T_n,Rejection Prop
,,,,,
Baseline,0.670$\pm$0.015,0.106$\pm$0.029,0.179$\pm$0.020,1.482$\pm$0.084,1.0
Project,0.659$\pm$0.004,0.064$\pm$0.006,0.199$\pm$0.005,1.077$\pm$0.017,0.0
Reduction,0.654$\pm$0.018,0.072$\pm$0.026,0.174$\pm$0.022,1.065$\pm$0.031,0.0
SenSR,0.625$\pm$0.048,0.037$\pm$0.032,0.183$\pm$0.076,1.021$\pm$0.019,0.0


In [12]:
print(res.to_latex(escape=False, column_format='l' + 'c'*len(colnames)))

\begin{tabular}{lccccc}
\toprule
{} &     Balanced Acc & $\text{AOD}_{\text{gen}}$ & $\text{AOD}_{\text{race}}$ &              T_n & Rejection Prop \\
{} \\
\midrule
Baseline  &  0.670$\pm$0.015 &           0.106$\pm$0.029 &            0.179$\pm$0.020 &  1.482$\pm$0.084 &            1.0 \\
Project   &  0.659$\pm$0.004 &           0.064$\pm$0.006 &            0.199$\pm$0.005 &  1.077$\pm$0.017 &            0.0 \\
Reduction &  0.654$\pm$0.018 &           0.072$\pm$0.026 &            0.174$\pm$0.022 &  1.065$\pm$0.031 &            0.0 \\
SenSR     &  0.625$\pm$0.048 &           0.037$\pm$0.032 &            0.183$\pm$0.076 &  1.021$\pm$0.019 &            0.0 \\
\bottomrule
\end{tabular}

